In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import mne
from scipy import signal
from scipy.signal import butter, filtfilt, iirnotch, detrend
from definitions import CHAN_LOC
import pyxdf  # For loading .xdf files

In [3]:
def load_xdf_file(filepath):
    """Load the XDF file using pyxdf."""
    streams, header = pyxdf.load_xdf(filepath)
    return streams, header

In [4]:
def perform_ICA(streams, num):
    # Step 2: Load your EEG data from the stream
    if num == 1:
        data = streams[0]['time_series'][:9, :]
        sampling_rate = streams[0]['info']['effective_srate']
    else:
        data = streams[1]['time_series'][:9, :]
        sampling_rate = streams[1]['info']['effective_srate']

    # Step 3: Load your channel locations file
    chanlocs = CHAN_LOC
    
    # Step 4: Create an MNE RawArray object from the data
    num_channels = data.shape[0]
    ch_names = ['EEG' + str(i) for i in range(1, num_channels + 1)]  # Assuming generic channel names
    montage = mne.channels.read_custom_montage(chanlocs)  # Load channel locations
    info = mne.create_info(ch_names=ch_names, sfreq=sampling_rate, ch_types='eeg')
    raw = mne.io.RawArray(data, info)
    raw.set_montage(montage)

    # Step 5: Preprocess the data (bandpass filter between 1-50 Hz)
    raw.filter(1., 50., fir_design='firwin')

    # Step 6: Run ICA
    ica = mne.preprocessing.ICA(n_components=num_channels, method='fastica', random_state=97)
    ica.fit(raw)

    # Optional: Save the ICA results
    ica.save('yourdata_ica.fif')  # Uncomment and modify the path to save the ICA

    return raw, ica

In [5]:
def filtering(EEG_data):
    # Sampling rate
    sampling_rate = EEG_data.info['sfreq']

    # Butterworth bandpass filter between 0.3 and 10 Hz (4th order)
    b_butter, a_butter = butter(4, [0.3, 10], btype='bandpass', fs=sampling_rate)

    # Mu wave frequency range
    mu_range = [8, 12]
    b_mu, a_mu = butter(4, mu_range, btype='bandpass', fs=sampling_rate)

    # Notch filter for power line noise at 50 Hz
    b_notch, a_notch = iirnotch(50, 50 / 35, sampling_rate)

    # Initialize the filtered data
    eeg_data_notch = np.zeros_like(EEG_data.get_data())
    eeg_data_but = np.zeros_like(EEG_data.get_data())
    eeg_data_mu = np.zeros_like(EEG_data.get_data())
    eeg_data_mean = np.zeros_like(EEG_data.get_data())

    # Apply filtering to each channel
    for i in range(EEG_data.get_data().shape[0]):
        # Apply notch filter to remove 50 Hz power line noise
        eeg_data_notch[i, :] = filtfilt(b_notch, a_notch, EEG_data.get_data()[i, :])

        # Apply Butterworth bandpass filter (0.3-10 Hz)
        eeg_data_but[i, :] = filtfilt(b_butter, a_butter, detrend(eeg_data_notch[i, :]))

        # Apply Butterworth filter for mu waves (8-12 Hz)
        eeg_data_mu[i, :] = filtfilt(b_mu, a_mu, detrend(eeg_data_but[i, :]))

        # Calculate the mean for each channel after mu wave filtering
        eeg_data_mean[i, :] = np.mean(eeg_data_mu[i, :])

    # Transpose the filtered data
    eeg_data_but = eeg_data_but.T

    # Zero-mean the data
    channels_mean = np.mean(eeg_data_but, axis=1)
    channels_mean_matrix = np.tile(channels_mean[:, np.newaxis], (1, eeg_data_but.shape[1]))
    EEG_filt = eeg_data_but - channels_mean_matrix

    return EEG_filt

In [6]:
def plot_eeg(EEG_filt, events, sampling_rate, title):
    """ Plot EEG data with event markers. """
    time_vector = np.arange(len(EEG_filt)) / sampling_rate
    plt.figure()
    plt.plot(time_vector, EEG_filt[:, 5])  # Cz channel (index 5)
    plt.xlabel('Time (s)')
    plt.ylabel('EEG Data')
    plt.title(title)
    for event in events:
        plt.axvline(x=event / sampling_rate, color='r', linestyle='--')
    plt.show()

In [7]:
def plot_topoplot(mean_data, title):
    """ Plot the topographic map (topoplot) of EEG data. """
    mne.viz.plot_topomap(mean_data, pos=CHAN_LOC, show=True)
    plt.title(title)
    plt.show()


In [10]:
# Load XDF files
xdfFilePath_Right = r".\DATASET\SUB_2\RIGHT\sub-P001\ses-S002\eeg\sub-P001_ses-S002_task-Default_run-001_eeg.xdf"
xdfFilePath_Left = xdfFilePath_Left = r".\DATASET\SUB_2\LEFT\sub-P001\ses-S002\eeg\sub-P001_ses-S002_task-Default_run-001_eeg.xdf"

streams_R, header_R = load_xdf_file(xdfFilePath_Right)
streams_L, header_L = load_xdf_file(xdfFilePath_Left)

# Perform ICA
EEG_DATA_R, ICA_R = perform_ICA(streams_R, 1)
EEG_DATA_L, ICA_L = perform_ICA(streams_L, 1)

# Filter EEG data
EEG_filt_R = filtering(EEG_DATA_R)
EEG_filt_L = filtering(EEG_DATA_L)

# Plot EEG Data with Event Markers
sampling_rate = EEG_DATA_R.info['sfreq']
events_right = streams_R[0]['time_stamps'] * sampling_rate
events_left = streams_L[0]['time_stamps'] * sampling_rate

plot_eeg(EEG_filt_R, events_right, sampling_rate, 'EEG Data with Event Markers (Right)')
plot_eeg(EEG_filt_L, events_left, sampling_rate, 'EEG Data with Event Markers (Left)')

# Create and plot Topoplots
mean_R_data = np.mean(EEG_filt_R, axis=0)
mean_L_data = np.mean(EEG_filt_L, axis=0)

plot_topoplot(mean_R_data, 'EEG TOPOPLOT for Right Events')
plot_topoplot(mean_L_data, 'EEG TOPOPLOT for Left Events')

Exception: file DATASET\SUB_2\RIGHT\sub-P001\ses-S002\eeg\sub-P001_ses-S002_task-Default_run-001_eeg.xdf does not exist.